In [24]:
import tensorflow as tf

class gru_model(tf.keras.Model):
    def __init__(self, rnn_units, num_layers):
        super(gru_model, self).__init__()
        self.num_layers = num_layers
        self.rnn_units = rnn_units
        self.grus = [tf.keras.layers.GRU(rnn_units[i],
                                   return_sequences=True, 
                                   return_state=True) for i in range(num_layers)]
        
        self.w1 = tf.keras.layers.Dense(10) ## Dense is a linear layer 

        self.w2 = tf.keras.layers.Dense(1) 


    def call(self, inputs, training=False):
        #input shape: (batch_size, seq_length, features)
        assert self.num_layers == len(self.rnn_units)
        x = inputs
        if (len(x.shape) != 3):
            print('what are you doing? this is wrong shape')
        print('input dim ({}, {}, {})'.format(x.shape[0], x.shape[1], x.shape[2]))
        whole_seq = x

        ## RNN stuff
        for i in range(self.num_layers):
            whole_seq, final_s = self.grus[i](whole_seq, training=training)
        ##
        
        ## extra layers
        target = self.w1(final_s)
        target = tf.nn.relu(target) ## just non-linearity 
        target = self.w2(target)
        
        ### The Dense layer right before the sigmoid must be dimension 1. 
        ### We must keep sigmoid for probability in [0, 1]
        target = tf.nn.sigmoid(target)
        
        return target

In [25]:
## if inputs are [x1, x2]
## Dense(x1, x2) == w1x1 + w2x2 

In [26]:
import tensorflow as tf
import math as m


def nll(y, y_pred):
    '''y_pred is a probability value between 0 to 1'''
    bce = tf.keras.losses.BinaryCrossentropy()    
    return bce(y, y_pred)


In [27]:

def build_graph():
    
    @tf.function(experimental_relax_shapes=True)
    def train_step(gru, optimizer, x,y, training=True):

        with tf.GradientTape(persistent=True) as tape:

            μ = gru(x, training) 
            xent = nll(y, μ)
        
        gradients = tape.gradient(xent, gru.trainable_variables)
        optimizer.apply_gradients(zip(gradients, gru.trainable_variables))
        return μ, xent

    tf.keras.backend.set_floatx('float32')
    return train_step

In [28]:
import numpy as np 

In [29]:
y = np.random.choice([0, 1], size = 32) 

In [30]:
array = np.random.normal(size = (32, 20, 1))

In [31]:
array.shape # batch_size X seq_len (timestamps in time-series) X features

(32, 20, 1)

In [32]:
g_model = gru_model([32], 1) #term in [] is size of hidden layers

In [33]:
g_model(array)
# output shape batch_size X 1 
# outptut is a probability between 0 to 1 
# interpretation is how likely each sequence to be a coral/not coral  

input dim (32, 20, 1)


<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.50750875],
       [0.50618815],
       [0.5028637 ],
       [0.5173028 ],
       [0.5000554 ],
       [0.5002938 ],
       [0.50272197],
       [0.51302785],
       [0.50136375],
       [0.4988638 ],
       [0.49182937],
       [0.51158327],
       [0.5231224 ],
       [0.5096445 ],
       [0.5024818 ],
       [0.503356  ],
       [0.49717882],
       [0.5076286 ],
       [0.50474733],
       [0.5098654 ],
       [0.49854904],
       [0.50177366],
       [0.49526474],
       [0.5010304 ],
       [0.5098653 ],
       [0.5176404 ],
       [0.50219506],
       [0.5025526 ],
       [0.4970097 ],
       [0.493574  ],
       [0.50689775],
       [0.50466263]], dtype=float32)>

In [34]:
nll(y, g_model(array))

input dim (32, 20, 1)


<tf.Tensor: shape=(), dtype=float32, numpy=0.69328344>

In [35]:
optimizer = tf.keras.optimizers.Adam(3e-4)

In [36]:
import tensorflow as tf

In [40]:
with tf.device("/CPU:0"):
    num_epochs = 1
    num_batches = 100
    tr_step = build_graph()
    for epoch in range(num_epochs):
        for batch in range(num_batches):
            array = np.random.normal(size = (32, 20, 1))# fake data
            y = np.random.choice([0, 1], size = 32)  # fake             
            ### array, y  = batcher_fun(data, training = True) shapes: (batch_s, seq_l, features), (batch_s, 1)
            y_pred, xent = tr_step(g_model, optimizer, array, y, training=True)
            
         ## validation set 
         ## test_set 

input dim (32, 20, 1)
input dim (32, 20, 1)


2023-05-05 18:15:34.210029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
